# Building a Spam Filter
In this project, I will study the practical side of the Naive Bayes algorithm by building a spam filter for SMS messages. 

The first task is to 'teach' the computer how to classify messages. We will use a Naive Bayes algorithm along with a dataset of 5,572 messages that are already classified by humans. 

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly [from this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection).

Let's start by reading in the dataset. 

In [1]:
import pandas as pd
data = pd.read_csv('SMSSpamCollection', sep = '\t', header = None,
                  names = ['Label', 'SMS'])
data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
data.shape

(5572, 2)

There are 5572 rows of the two columns within the dataset. 

In [3]:
data['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

ham = 'non-spam' and spam is obviously spam emails. So the majority of the dataset is labeled as non-spam. 

### Creating a Test
Before creating a filter, it's helpful to first think of a way to test how well it works. So first, let's split the dataset into two categories:
- A training set, which we'll use to train the computer how to classify messages. 
- A test set, which we'll use to test how good the spam filter is with classifying new messages. 

We're going to keep 80% of test for training, and 20% for testing/ The set has 5,572 messages, so sorted we will have:
- Training set will have 4,458 messages
- Test set will have 1,114 messages

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% - so we expect more than 80% of the new messages will be classified correctly as spam or ham. 

In [4]:
random = data.sample(random_state = 1, frac = 1)
training_set = random[:4458]
test_set = random[-1114:]

In [5]:
training_set = training_set.reset_index(drop = True)
test_set = test_set.reset_index(drop = True)

In [6]:
print('Full Dataset:\n',data['Label'].value_counts(normalize = True),'\n')
print('Training Set:\n',training_set['Label'].value_counts(normalize = True),'\n')
print('Testing Set:\n',test_set['Label'].value_counts(normalize = True))

Full Dataset:
 ham     0.865937
spam    0.134063
Name: Label, dtype: float64 

Training Set:
 ham     0.86541
spam    0.13459
Name: Label, dtype: float64 

Testing Set:
 ham     0.868043
spam    0.131957
Name: Label, dtype: float64


So we have a good estimate of the full dataset with our randomized datasets. 

### Transforming the Data
The next big step is to use the training set to teach the algorithm to classify new messages. We'll use a Naive Bayes algorithm to make the classification. 


To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. We will need to transform the data into a format where:
- The SMS column doesn't exist
- Instead, the SMS column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values spam, 2, 2, 1, 1, 0, 0, 0, 0, 0. These values tell us that:
    - The message is spam. 
    - The word 'secret' occurs two times inside the message
    - The word "prize" occurs two times inside the message.
    - The word "claim" occurs one time inside the message.
    - The word "now" occurs one time inside the message.
    - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

| Label | secret | prize | claim | now | coming | to | my | party | winner
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| spam | 2 | 2 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| ham | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 |
| spam | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 |

In [7]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


With the exception of the 'label' column, every other column in the transformed table we're aiming to make represents a unique word. With this, we can create a vocabulary set/list to start making these columns. 

In [9]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary)) #To remove duplicates

In [10]:
training_set.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [11]:
len(vocabulary)

7783

There are 7783 unique words in this dataset.

### Final Training Set
We're now going to make the dataset transformation.

In [26]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [27]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [14]:
clean = pd.concat([training_set, word_counts], axis=1)
clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


### Creating a Spam Filter
Recall that Naive Bayes algorithm needs to know the probability values of P(wi|Spam) & P(wi|Ham). Let's start with calculating p(spam) and p(ham)...

In [15]:
p_spam = len(clean[clean['Label'] == 'spam']) / len(clean)
p_ham = len(clean[clean['Label'] == 'ham']) / len(clean)

In [16]:
print(p_spam)
print(p_ham)

0.13458950201884254
0.8654104979811574


Now let's calculate Nspam, Nham, and Nvocabulary
We need to remember that:
- Nspam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.
- Nham is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.

We'll use laplace smoothing and set alpha = 1.

In [17]:
#Initialize list for spam and ham
spam_list = []
ham_list = []

#Isolating spam and ham
spam_only = clean[clean['Label'] == 'spam']
ham_only = clean[clean['Label'] == 'ham']

#Append to lists
for sms in spam_only['SMS']:
    for word in sms:
        spam_list.append(word)
        
for sms in ham_only['SMS']:
    for word in sms:
        ham_list.append(word)
        
n_vocabulary = len(vocabulary)
n_spam = len(spam_list)
n_ham = len(ham_list)

#Laplace Smoothing Set-Up
alpha = 1

### Creating Parameters
We just calculated a few terms:
- P(spam) and P(ham)
- Nspam, Nham, Nvocabulary

These terms will have constant values in application for every message. However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words.

In [18]:
#Establishing dictionaries for each word
parameters_spam = {unique_word : 0 for unique_word in vocabulary}
parameters_ham = {unique_word : 0 for unique_word in vocabulary}

#Calculating probability of calculate P(wi|Spam) and P(wi|Ham)
for word in vocabulary:
    n_word_given_spam = spam_only[word].sum()
    p_w_spam = (n_word_given_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    parameters_spam[word] = p_w_spam
    
    n_word_given_ham = ham_only[word].sum()
    p_w_ham = (n_word_given_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    parameters_ham[word] = p_w_ham   

### Creating Spam Filter
The spam filter can be understood as a function that:
- Takes in as input a new message
- Calculates probability of message being spam, or ham, given the words in the message. 
- Compares values of such probabilities, and assigns a label depending on which one is greater. 
    - If the two are equal, the algorithm may request human help. 


In [19]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:
    '''
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

We can test this with some obvious spam message versus a typical email response.

In [20]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [21]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Test Set
Now we can test this function with the test set of 1,114 messages. 

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them.

In [22]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [23]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

In [24]:
#Measuring Accuracy

#Intialize variables
correct = 0
total = len(test_set['SMS'])

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = (correct / total)
incorrect = total - correct

print('''The accuracy of our algorithm is: {:%}, with {} correct 
values and {} incorrect values.'''.format(accuracy, correct, incorrect))

The accuracy of our algorithm is: 98.743268%, with 1100 correct 
values and 14 incorrect values.


# Conclusion
Through this project we were able to create a Naive Bayes algorithm to analyze messages and assign them a label of 'spam' or 'ham' (not spam). We sliced and transformed our training dataset to establish probabilities for spam and ham messages, then applied such numbers towards an algorithm to label custom messages and the remainer of our dataset, the testing set. 

Our model had a **98.7%** accuracy when we applied it to our testing set, which indicates a highly accurate model that could be used for more datasets.